####This notebook analyzes the 



In [1]:
import os, sys,z5py
import numpy as np
import pandas as pd
from glob import glob 
from skimage.measure import regionprops
from skimage.io import imread, imsave
from os.path import abspath, dirname
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('default')
from scipy import stats, spatial
import seaborn as sns
from sklearn.preprocessing import binarize,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.utils import shuffle
from sklearn.inspection import permutation_importance


In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.

###### Load data files

In [2]:
lha_neuron=pd.read_csv('/Users/wangy6/Desktop/FISH/LHA_FISH_gene.csv',sep=',', index_col=0)
ex_m=pd.read_csv('/Users/wangy6/Desktop/LHA_figures/Slc17a6_metadata_tsne_v6.csv',sep=',', index_col=0)
inh_m=pd.read_csv('/Users/wangy6/Desktop/LHA_figures/Slc32a1_metadata_tsne_v6.csv',sep=',', index_col=0)
lha_neuron=lha_neuron.T
ex_m.x='Ex_' + ex_m.x.astype(str)
inh_m.x='Inh_' + inh_m.x.astype(str)
lha_neuron=lha_neuron.where(lha_neuron>=0, 0)


cluster=pd.concat([ex_m,inh_m],axis=0)

c=['Ex_1', 'Ex_2', 'Ex_3', 'Ex_4',
       'Ex_5', 'Ex_6', 'Ex_7', 'Ex_8', 'Ex_9', 'Ex_10', 'Ex_11', 'Ex_12', 'Ex_13', 'Ex_14', 
       'Ex_15', 'Ex_16', 'Ex_17', 'Ex_18', 'Ex_19', 'Ex_20',
       'Ex_21', 'Ex_22', 'Ex_23', 'Ex_24', 'Ex_25','Inh_1', 'Inh_2','Inh_3', 'Inh_4', 'Inh_5', 'Inh_6', 'Inh_7', 'Inh_8', 'Inh_9', 'Inh_10', 'Inh_11', 'Inh_12', 'Inh_13', 'Inh_14',
       'Inh_15', 'Inh_16', 'Inh_17', 'Inh_18', 'Inh_19','Inh_20', 'Inh_21', 'Inh_22', 'Inh_23']

roi=pd.read_csv('/Volumes/multifish/Yuhan/LHA_analysis/anatomy/20201007_roi_rigid_transformed_meta_v9_in_microns.csv',sep=',', index_col=0)

In [3]:
lha_neuron=pd.read_csv('directory/spotcount/neuron',sep=',', index_col=0)
ex_m=pd.read_csv('/Slc17a6/molecular/type/metadata',sep=',', index_col=0)
inh_m=pd.read_csv('/Slc32a1/molecular/type/metadata',sep=',', index_col=0)
lha_neuron=lha_neuron.T
lha_neuron=lha_neuron.where(lha_neuron>=0, 0)
roi=pd.read_csv('directory/roi/metadata',sep=',', index_col=0)

cluster=pd.concat([ex_m,inh_m],axis=0)

c=['Ex-1', 'Ex-2', 'Ex-3', 'Ex-4',
   'Ex-5', 'Ex-6', 'Ex-7', 'Ex-8', 
   'Ex-9', 'Ex-10', 'Ex-11', 'Ex-12', 
   'Ex-13', 'Ex-14', 'Ex-15', 'Ex-16', 
   'Ex-17', 'Ex-18', 'Ex-19', 'Ex-20',
   'Ex-21', 'Ex-22', 'Ex-23', 'Ex-24', 
   'Ex-25','Inh-1', 'Inh-2','Inh-3', 'Inh-4', 
   'Inh-5', 'Inh-6', 'Inh-7', 'Inh-8', 'Inh-9', 
   'Inh-10', 'Inh-11', 'Inh-12', 'Inh-13', 'Inh-14',
   'Inh-15', 'Inh-16', 'Inh-17', 'Inh-18', 'Inh-19',
   'Inh-20', 'Inh-21', 'Inh-22', 'Inh-23']

###### Data transformation

In [3]:
FISH_z=stats.zscore(lha_neuron)
fish=pd.DataFrame(data=FISH_z, index=lha_neuron.index, columns=lha_neuron.columns)
FISH_log=np.log(lha_neuron+1)
fish_log=pd.DataFrame(data=FISH_log, index=lha_neuron.index, columns=lha_neuron.columns)

log_z=stats.zscore(FISH_log)
log_z=pd.DataFrame(data=log_z, index=lha_neuron.index, columns=lha_neuron.columns)



###### Split data into training data and test data by random sampling

In [4]:

##predict x, y, z position with gene expression data + other features
roi_con=roi.copy()
roi_con=roi_con[(roi_con.x<777)&(roi_con.y<772)&(roi_con.z<266)]
roi_con=roi_con[roi_con.index.str.contains('LHA5')]
n=5000
idx=np.random.choice(roi_con.index,n)
X_train=log_z[log_z.index.isin(idx)]
Y_train=roi_con[roi_con.index.isin(idx)].to_numpy()[:, :3]
idx_t=np.random.choice(roi_con[~roi_con.index.isin(idx)].index, n)
X_test=log_z[log_z.index.isin(idx_t)]
Y_test=roi_con[roi_con.index.isin(idx_t)].to_numpy()[:, :3]



###### Multi-Output Random forest regression

In [5]:
max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=0, oob_score=True))
regr_multirf.fit(X_train, Y_train)
print('Random Forest regression prediction score: ', regr_multirf.score(X_test, Y_test))

Random Forest regression prediction score:  0.5960931938539519


In [7]:
from sklearn import model_selection
seed = 7
SS = model_selection.ShuffleSplit(n_splits=10)

scoring = 'explained_variance'
regr_multirf.fit(X_train, Y_train)
results = model_selection.cross_val_score(regr_multirf, X_test, Y_test,cv=SS, scoring=scoring)
print(f"explained variance (ShuffleSplit)  "
              f"{results.mean():.2f}"
              f" +/- {results.std():.2f}")

explained variance (ShuffleSplit)  0.60 +/- 0.02


In [8]:
from sklearn import model_selection
seed = 7
SS = model_selection.KFold(10)

scoring = 'explained_variance'
regr_multirf.fit(X_train, Y_train)
results = model_selection.cross_val_score(regr_multirf, X_test, Y_test,cv=SS, scoring=scoring)
print(f"explained variance (10-fold)  "
              f"{results.mean():.2f}"
              f" +/- {results.std():.2f}")

explained variance (10-fold)  0.58 +/- 0.03


In [9]:
# a=pd.DataFrame(np.empty([len(X_train.columns),0]), index=X_train.columns)
# for x in X_train.columns:
X1=X_train.drop(columns=['Otp','Meis2'])
X2=X_test.drop(columns=['Otp','Meis2'])
regr_multirf.fit(X1, Y_train)
SS = model_selection.ShuffleSplit(n_splits=10)
scoring = 'explained_variance'
results = model_selection.cross_val_score(regr_multirf, X2, Y_test,cv=SS, scoring=scoring)
print(f"explained variance (-Otp/Meis2)  "
              f"{results.mean():.2f}"
              f" +/- {results.std():.2f}")

explained variance (-Otp/Meis2)  0.57 +/- 0.02


In [10]:
X1=X_train.drop(columns=['Slc17a6','Slc32a1'])
X2=X_test.drop(columns=['Slc17a6','Slc32a1'])
regr_multirf.fit(X1, Y_train)
SS = model_selection.ShuffleSplit(n_splits=10)
scoring = 'explained_variance'
results = model_selection.cross_val_score(regr_multirf, X2, Y_test,cv=SS, scoring=scoring)
print(f"explained variance(-Slc17a6/Slc32a1)   "
              f"{results.mean():.2f}"
              f" +/- {results.std():.2f}")

explained variance(-Slc17a6/Slc32a1)   0.56 +/- 0.02


In [13]:
# a=pd.DataFrame(np.empty([len(X_train.columns),0]), index=X_train.columns)
# for x in X_train.columns:
X1=X_train.drop(columns=['Slc17a6','Slc32a1','Otp','Meis2'])
X2=X_test.drop(columns=['Slc17a6','Slc32a1','Otp','Meis2'])
regr_multirf.fit(X1, Y_train)
SS = model_selection.ShuffleSplit(n_splits=10)
scoring = 'explained_variance'
results = model_selection.cross_val_score(regr_multirf, X2, Y_test,cv=SS, scoring=scoring)
print(f"explained variance(-Otp/Meis2/Slc17a6/Slc32a1)  "
              f"{results.mean():.2f}"
              f" +/- {results.std():.2f}")

explained variance(-Otp/Meis2/Slc17a6/Slc32a1)  0.54 +/- 0.02


In [25]:
shuffle=[]
for i in range(0,1001):
    roi
    idx=np.random.choice(roi_con.index,n)
    X_train=log_z[log_z.index.isin(idx)]
    Y_train=roi_con[roi_con.index.isin(idx)].sample(frac = 1).to_numpy()[:, :3]
    idx_t=np.random.choice(roi_con[~roi_con.index.isin(idx)].index, n)
    X_test=log_z[log_z.index.isin(idx_t)]
    Y_test=roi_con[roi_con.index.isin(idx_t)].sample(frac = 1).to_numpy()[:, :3]
    regr_multirf.fit(X_train, Y_train)
    shuffle=np.append(shuffle, regr_multirf.score(X_test, Y_test))


In [38]:
from sklearn.dummy import DummyRegressor
clf = DummyRegressor(strategy='mean')
clf.fit(X_train, Y_train)
DummyRegressor(strategy='mean')
clf.score(X_test, Y_test)

-0.0003640707039280766

In [28]:
a=pd.DataFrame(np.empty([len(X_train.columns),0]), index=X_train.columns)
for x in X_train.columns:
    X1=X_train.drop(columns=x)
    X2=X_test.drop(columns=x)
    regr_multirf.fit(X1, Y_train)
    results = model_selection.cross_val_score(regr_multirf, X2, Y_test, cv=SS, scoring=scoring)
    a.loc[x,'explained_variance']=results.mean()
    a.loc[x,'std']=results.std()
    

In [33]:
sorted_idx = a['R^2 score'].argsort()[::-1] 

y_ticks = np.arange(0, len(a.index))
fig, ax = plt.subplots()
ax.barh(y_ticks, a['R^2 score'][sorted_idx])
ax.set_yticklabels(a.index[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest R^2 score (leave out selected feature)")
plt.xlim(0.4, 0.6)
fig.tight_layout()
plt.show()

In [59]:
df=pd.DataFrame(data=np.empty([len(X_train.columns),0]), index=X_train.columns)
df['feature_z']=regr_multirf.estimators_[0].feature_importances_
df['feature_y']=regr_multirf.estimators_[1].feature_importances_
df['feature_x']=regr_multirf.estimators_[2].feature_importances_
#df.sort_values(['feature_z', 'feature_y', 'feature_x'], ascending=False)

# sorted_idx = df['feature_z'].argsort()
# y_ticks = np.arange(0, len(feature_names))
# fig, ax = plt.subplots()
# ax.barh(y_ticks, df['feature_z'][sorted_idx])
# ax.set_yticklabels(feature_names[sorted_idx])
# ax.set_yticks(y_ticks)
# ax.set_title("Random Forest Feature Importances (MDI)")
# fig.tight_layout()
# plt.show()

In [30]:
r = permutation_importance(regr_multirf, X_test, Y_test,n_repeats=30,random_state=0)


In [ ]:
r_x = permutation_importance(regr_multirf.estimators_[2], X_test, Y_test[:,2],n_repeats=30,random_state=0)

In [ ]:
r_y = permutation_importance(regr_multirf.estimators_[1], X_test, Y_test[:,1],n_repeats=30,random_state=0)
r_z = permutation_importance(regr_multirf.estimators_[0], X_test, Y_test[:,0],n_repeats=30,random_state=0)

In [ ]:
df['feature_permu_z']=r_z['importances_mean']
df['feature_permu_y']=r_y['importances_mean']
df['feature_permu_x']=r_x['importances_mean']

In [ ]:
sorted_idx = df.feature_mean.argsort()

y_ticks = np.arange(0, len(df.index))
fig, ax = plt.subplots()
ax.barh(y_ticks, df.feature_mean[sorted_idx])
ax.set_yticklabels(df.index[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

In [ ]:
sorted_idx = r.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(r.importances[sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
ax.set_title("Feature Importance")
fig.tight_layout()
plt.show()